In [1]:
from langchain.chains import ConversationalRetrievalChain
from flask import Flask, render_template,jsonify,request
from flask_cors import CORS
from flask import Flask, render_template, url_for
import requests,openai,os

from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

from langchain_core.prompts import PromptTemplate


In [2]:
#memory = ConversationBufferMemory()


In [3]:


#DATA_PATH = "data/dataset"


app = Flask(__name__,static_folder='static',template_folder='templates')
CORS(app)


import sys
print(sys.path)



embeddings = OpenAIEmbeddings()

#vectorstore_from_docs = PineconeVectorStore.from_existing_index(index_name="michaeles", embedding=embeddings)

embeddings = OpenAIEmbeddings()

pc = Pinecone(api_key='9619b277-b411-43a6-be65-689f4fa2907e')
print(pc.list_indexes())
index = pc.Index('michaeles')
print(index)

vectorstore = PineconeVectorStore(index=index, embedding=embeddings)


['E:\\FIVER\\sardarali\\projects\\mike_RAG\\adv\\RAG_Processed\\flask\\final_app', 'C:\\Users\\qaise\\Anaconda3\\envs\\tf_1_15', 'C:\\Users\\qaise\\Anaconda3\\envs\\pf2\\python310.zip', 'C:\\Users\\qaise\\Anaconda3\\envs\\pf2\\DLLs', 'C:\\Users\\qaise\\Anaconda3\\envs\\pf2\\lib', 'C:\\Users\\qaise\\Anaconda3\\envs\\pf2', '', 'C:\\Users\\qaise\\Anaconda3\\envs\\pf2\\lib\\site-packages', 'C:\\Users\\qaise\\Anaconda3\\envs\\pf2\\lib\\site-packages\\win32', 'C:\\Users\\qaise\\Anaconda3\\envs\\pf2\\lib\\site-packages\\win32\\lib', 'C:\\Users\\qaise\\Anaconda3\\envs\\pf2\\lib\\site-packages\\Pythonwin']
{'indexes': [{'dimension': 1536,
              'host': 'michaeles-2j7mz6k.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'michaeles',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [12]:
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
)

In [13]:
llm = ChatOpenAI(temperature=0.5, model_name="gpt-4o")
#memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key='answer')

In [14]:

template = """Answer the question in your own words..
If you do not know the answer to the question, simply respond with "I don't know. Can you ask another question", and your response should be friendly.
If questions are asked where there is no relevant context available, simply respond with "I can't find this information in given documents, Can you please repeat your question by providing more information."
Here is Contex: {context}


{chat_history}
Human: {question}
Assistant:"""

prompt = PromptTemplate(
    input_variables=["context", "chat_history", "question"], template=template
)



In [17]:
# Create the multipurpose chain
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key='answer')
qachat = ConversationalRetrievalChain.from_llm(
        llm=llm, retriever=retriever, memory=memory,
        return_source_documents=True,
        combine_docs_chain_kwargs={'prompt': prompt}
)

ans = qachat("whats are learning outcomes in chapter 5?")['answer']
ans

"Chapter 5's learning outcomes include:\n\n1. Defining the team's playstyle and refining team strategies.\n2. Enhancing teamwork leading up to tournaments.\n3. Learning multiple stress management techniques to handle competitive gaming pressure.\n4. Demonstrating a deeper understanding of Rocket League tournament formats and rules.\n\nIf you have any other questions or need further details, feel free to ask!"

In [18]:
ans = qachat("whats are learning outcomes in chapter 5?")['answer']
ans

"The learning outcomes in Chapter 5 are:\n\n1. Defining the team's playstyle and refining team strategies.\n2. Enhancing teamwork leading up to tournaments.\n3. Learning multiple stress management techniques to cope with the pressure of competitive gaming.\n4. Demonstrating a deeper understanding of Rocket League tournament formats and rules.\n\nIf you have any other questions or need further details, feel free to ask!"

In [18]:
qachat("I mean explain everything thats mentioned in it?")['answer']

'Sure, here are the learning outcomes mentioned in Chapter 2 "Roles and Rules":\n\n1. **Effective Positioning Strategies**: Students will be able to demonstrate effective positioning strategies in various gameplay situations by understanding the importance of specific roles. This means students will learn how to position themselves correctly on the field based on whether they are playing offense, defense, or support in Rocket League.\n\n2. **Understanding Key Principles in Ethics**: Students will be able to understand key principles in ethics including integrity, fair play, and respect while applying these principles to real-world scenarios in the context of competitive gaming. This involves grasping the importance of honesty, strong moral principles (integrity), respecting rules and demonstrating good sportsmanship (fair play), and treating everyone with courtesy and positivity (respect).\n\n3. **Importance of Effective Communication**: Students will be able to grasp the importance of

In [11]:
#qachat("anything else important that you forgot to mentioned?")['answer']

'Based on the provided context, it seems like the key points related to physical conditioning, mental wellbeing, stress management, nutrition, hydration, and team dynamics have been covered. If there are any specific aspects or topics you are looking for, please let me know so I can provide more information.'

In [12]:
#qachat("what are ethics in esports?")['answer']

'Ethics in esports refer to the principles and values that guide the behavior of players, organizers, sponsors, and fans in the world of competitive gaming. This includes concepts such as integrity, fair play, and respect, which are essential for maintaining a positive and healthy gaming community.'

In [13]:
#qachat("can you explain this sponsors?")['answer']

"I don't have specific information on the role of sponsors in esports based on the provided context."

In [14]:
#qachat("whats in chapter 8?")['answer']

'Chapter 8 covers Strategy Crafting, where students learn about developing and refining strategies for gameplay. It emphasizes role collaboration, development, and improvement for personal and team growth. The facilitator is encouraged to provide guidance, correct and advise students as needed, and ensure active in-class participation. Students are also reminded to complete the ecourse prior to the next class and submit post-class assessments on time.'

In [15]:
#qachat("explain this in 30 lines.")['answer']

"In Chapter 8 on Strategy Crafting in esports, the main focus is on the importance of team roles within an esports team. The chapter covers the four main roles essential for an esports team: Coach, Physical Trainer, Sports Psychologist, and Social Media Manager. Students learn about the roles, responsibilities, and team management principles associated with each role. The chapter guides students in selecting and assigning specific roles within their teams and emphasizes the significance of specialized roles in achieving team success.\n\nKey concepts covered in this chapter include the integration of roles into team strategy, maximizing team performance through role synergy, and developing the necessary skills for effective collaboration and team management. The chapter aims to deepen students' understanding of leadership, team management, and the importance of specialized roles in esports teams.\n\nOne of the activities included in this chapter is the Team Strategy Integration Challeng

In [16]:
qachat("Hi, can you tell me whats title of chapter 6 document?")['answer']

'The title of the document for Chapter 6 is "Trainer Assessment Form."'

In [17]:
qachat("Hi, can you tell me whats title of chapter 6 document?")['answer']

'The title of the document for Chapter 6 is "Trainer Assessment Form."'

In [11]:
test = qachat("what learning outcome in chapter 3")

In [12]:
test['chat_history']

[HumanMessage(content='whats in chapter 6?'),
 AIMessage(content="I don't have specific information on the contents of Chapter 6 as the provided context does not include details about the content of that chapter."),
 HumanMessage(content='whats in chapter 6?'),
 AIMessage(content='Chapter 6 is titled "Physical Conditioning" and focuses on physical fitness tailored for esports athletes. It includes an eCourse covering physical training modules, recommendations on proper nutrition, and tips for maintaining mental well-being for gamers. In the in-person session, students engage in a pre-game group fitness routine to highlight the benefits of physical fitness for improving gaming performance. The chapter emphasizes integrating the physical exercises learned into daily routines to experience the positive impact of physical conditioning on gaming abilities.'),
 HumanMessage(content='I mean explain everything thats mentioned in it?'),
 AIMessage(content='Chapter 6 titled "Physical Conditionin

In [3]:
from openai import OpenAI

In [4]:
####### put openai client here
MODEL="gpt-4o"

In [25]:
completion = client.chat.completions.create(
  model=MODEL,
  messages=[
    {"role": "system", "content": ""}, #You are a helpful assistant and your task is to provide refine output minimal way possible of first language model that is trained on custom data, you are second language model in this chain. first language model sometime produces silly response so you have to correct it if needed, If you recieve something like I don't know, or I can't find information, then you have to answer that question according to your understanding and on base of chat history that I will provide.
    {"role": "user", "content": "i don't know, chat history : {}"}
  ]
)
print("Assistant: " + completion.choices[0].message.content)

Assistant: Could you please specify what you need help with? I'll do my best to assist you.


In [8]:
completion = client.chat.completions.create(
  model=MODEL,
  messages=[
    {"role": "system", "content": "you are helpful assistant"}, #You are a helpful assistant and your task is to provide refine output minimal way possible of first language model that is trained on custom data, you are second language model in this chain. first language model sometime produces silly response so you have to correct it if needed, If you recieve something like I don't know, or I can't find information, then you have to answer that question according to your understanding and on base of chat history that I will provide.
        {"role": "user", "content": ""}
  ]
)
print("Assistant: " + completion.choices[0].message.content)

Assistant: I'm sorry, but I can't assist with that.


In [18]:
import pandas as pd
df = pd.DataFrame({
    'user query':[],
    'date':[]
})
df.to_csv('history.csv')

In [19]:
import datetime

In [20]:
current_date = datetime.datetime.now()
formatted_date = current_date.strftime('%Y-%m-%d')


In [21]:
df = pd.read_csv('history.csv')
current_date = datetime.datetime.now()
formatted_date = current_date.strftime('%Y-%m-%d')
#df = pd.concat([df, pd.DataFrame(['talagang is good',formatted_date])], ignore_index=True)
#print(df)

In [22]:
df

,Unnamed: 0,user query,date


In [23]:
df.loc[len(df)]=['','talagang is good',formatted_date]

In [24]:
df

,Unnamed: 0,user query,date
0,,talagang is good,2024-05-26
